# TAPE: evaluation demo 

See how to:
- Evaluate your model on TAPE
- Report model performance on chosen tasks

## Model Evaluation

### Install

In [1]:
# !git clone https://github.com/RussianNLP/TAPE
# %cd TAPE
# !pip install .

### Load Modules

In [1]:
import pandas as pd
import numpy as np
from datasets import load_dataset
from TAPE.utils.episodes import get_episode_data

/home/kosenko/miniconda3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Read Data

We will be working with the **RuWorldTree**, a multiple choice QA task:

In [2]:
data = load_dataset("RussianNLP/tape", "ru_worldtree.episodes")
train_data = data["train"].data.to_pandas()
test_data = data["test"].data.to_pandas()

In [3]:
train_data.head()

,question,exam_name,school_grade,knowledge_type,answer,perturbation,episode
0,"Тунец - это океаническая рыба, которая хорошо ...",MCAS,5,"CAUSAL,MODEL",A,ru_worldtree,"[10, 11]"
1,Какая часть растения больше всего отвечает за ...,MCAS,5,"PROP,PROCESS",B,ru_worldtree,[11]
2,Тамара купила скейт и комплект для изготовлени...,MCAS,5,"CAUSAL,MODEL",D,ru_worldtree,"[15, 16]"
3,"Кальмар - животное, обитающее в океане. Он выт...",MCAS,5,MODEL,C,ru_worldtree,"[18, 19]"
4,Цвет глаз - это физическая черта. Какое утверж...,Maryland School Assessment - Science,4,"MODEL,PROP",B,ru_worldtree,"[15, 17]"


In [10]:
test_data

,question,exam_name,school_grade,knowledge_type,answer,perturbation,episode
0,Что из следующего является примером формы энер...,MCAS,5,EXAMPLE,None,ru_worldtree,[]
1,Дима проверил физические свойства минерала. Он...,MCAS,5,MODEL,None,ru_worldtree,[]
2,"В течение большей части года воздух над Сочи, ...",MCAS,5,MODEL,None,ru_worldtree,[]
3,Какое из следующих утверждений лучше всего объ...,MCAS,5,EXAMPLE,None,ru_worldtree,[]
4,"Кипрей болотный - растение, лучше всего растущ...",MCAS,5,MODEL,None,ru_worldtree,[]
...,...,...,...,...,...,...,...
4398,разные Есть виды пустыни . Что у них общего ? ...,TIMSS,4,NO TYPE,None,swap,[]
4399,"Алина измерила , сколько сахара растворяется с...",TIMSS,4,NO TYPE,None,swap,[]
4400,Растения используют энергию непосредственно от...,TIMSS,4,NO TYPE,None,swap,[]
4401,Что следующего описывает конденсацию ? (A) тел...,TIMSS,4,NO TYPE,None,swap,[]


Original test with no perturbations:

In [ ]:
test_data[test_data.perturbation == "ru_worldtree"]

Test data with `BackTranslation` perturbation:

In [ ]:
test_data[test_data.perturbation == "back_translation"]

### Model prediction

To evaluate your model on TAPE, first, create a function for model prediction. Here we have a simple random model:

In [4]:
def evaluate_model(k_shots, test_data):
    predictions = np.random.choice(["A", "B", "C", "D"], size=test_data.shape[0])
    return predictions

### Predict

The evaluation loop looks as follows: each episode corresponds to `k` samples from the train data, used as shots, the model is then evaluated on the test data and its perturbed versions.

For a more detailed explanation of episodes refer to [notebooks/episode_example.ipynb](https://github.com/RussianNLP/TAPE/notebooks/episode_example.ipynb).

In [5]:
# iterate over episodes
evaluation_results = []
for episode in sorted(train_data.episode.apply(lambda x: x[0]).unique()):

    k_shots = get_episode_data(train_data, episode)

    # iterate over transformed and original test datasets
    for perturbation, test in test_data.groupby("perturbation"):

        # get model predictions
        predictions = evaluate_model(k_shots, test)

        # save predictions
        evaluation_results.append(
            {
                "episode": episode,
                "shot": k_shots.shape[0],
                "slice": perturbation,
                "preds": predictions,
            }
        )

evaluation_results = pd.DataFrame(evaluation_results)
evaluation_results.head()

,episode,shot,slice,preds
0,5,1,addsent,"[C, A, A, A, A, B, D, D, B, D, C, B, A, D, C, ..."
1,5,1,back_translation,"[C, C, D, A, A, A, D, C, B, C, C, D, C, B, C, ..."
2,5,1,butter_fingers,"[C, D, A, C, C, C, D, B, D, C, B, D, A, C, B, ..."
3,5,1,del,"[B, D, D, B, A, B, A, A, A, B, D, C, C, B, B, ..."
4,5,1,emojify,"[A, A, A, D, C, A, A, C, D, D, C, B, A, C, C, ..."


In [9]:
k_shots['question'].tolist()

['Кальмар - животное, обитающее в океане. Он выталкивает из своего тела поток воды, заставляя его быстро двигаться. Какая из следующих механических систем движется наиболее похоже на кальмара? (A) автобус (B) вертолет (C) ракета (D) поезд',
 'Что лучше всего описывает роль пищевода в пищеварении? (A) Он выделяет кислоту и смешивает пищу. (B) Он помогает усваивать питательные вещества из пищи. (C) Он переносит пищу изо рта в желудок. (D) Он переносит пищу из желудка в кишечник.',
 'Какая форма энергии заставляет таять кубик льда? (A) механическая (B) магнитная (C) звуковая (D) тепловая',
 'Во время эксперимента студент сообщает, что жидкость стала зеленой при смешивании с другой жидкостью. Это пример (A) измерения (B) прогноза (C) объяснения (D) наблюдения',
 'Со временем у лисы отрастает густой мех. Эта адаптация помогает лисе (A) находить пищу (B) сохранять тепло (C) становиться сильнее (D) убегать от хищников',
 'Какое из следующих живых существ в системе прудов использует энергию со

## Reports

Now, when we have the predictions of our model, we can evaluate its performance.

**Note:**  We generate targets randomly to avoid data leakage. Submit your predictions to see true model performance on TAPE.

### Load Modules

In [9]:
from TAPE.reports import report_transformations, report_subpopulations, Report
from TAPE.subpopulations import LengthSubpopulation

### Preparing Data

To plot your evaluation report, create a `prepare_report` function. It should take the predictions of your model and evaluate it across several paradigms, including perturbations and subpopulations.

If your predictions are in the correct format (see above), you can simply use the function below.

In [10]:
# this file is only available to the TAPE authors
# but you can evaluate your model on the public sample, published in BigBench

test_answers = np.load("../TAPE_test_answers/episodes/worldtree_answers.npy")

FileNotFoundError: [Errno 2] No such file or directory: '../TAPE_test_answers/episodes/worldtree_answers.npy'

In [11]:
def prepare_report(
    predictions: pd.DataFrame,
    task_name: str,
    subpopulations: list,
    original_data: pd.DataFrame,
    label_ids: str = "label",
):

    # report perturbations
    transformation_res = report_transformations(
        predictions, task_name, label_ids
    )

    # report subpopulations
    sub_res = report_subpopulations(
        subpopulations=subpopulations,
        org_data=original_data,
        preds=predictions,
        dataset_name=task_name,
        label_ids=label_ids,
    )

    results = pd.concat([transformation_res, sub_res])
    return results

In [12]:
task_name = "ru_worldtree"
subpopulations = [LengthSubpopulation("question")]

# aggregate results
results = prepare_report(
    predictions=evaluation_results,  # model predictions
    task_name=task_name,  # name of the task
    original_data=train_data[
        train_data.perturbation == "ru_worldtree"
    ],  # original (not perturbed) data
    label_ids=test_answers,  # targets for the original test set
    subpopulations=subpopulations,  # list of subpopulations to analyse
)
results

NameError: name 'test_answers' is not defined

Now we have our aggregated results in the correct format, with the following infromation:

- `shot`: number of shots used for evaluation
- `slice`: slice name to apper in the report (either a perturbation or a subpopulation)
- `size`: size of the slice
- `asr`: attack success rate score (percentage of correct predictions that changed to incorrect after perturbation)
- `macro_f1`: macro f1 scores / token overlap (SQuAD F1)
- `accuracy`: accuracy / exact match scores
- `std_<x>`: standard deviations  over the episodes for metric x
- `category`: type of the slice (evalset, perturbations, subpopulation)

Now we can plot the report:

In [13]:
report = Report(results)
report.figure(shot=4)

NameError: name 'results' is not defined

(The reports do not always render in the notebooks, so we attach an example image bellow)
![eval_report](https://github.com/RussianNLP/TAPE/blob/main/images/eval_report_example.png?raw=true)